In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np
import pandas as pd

In [2]:
STORM_NUMBER = 3
TITLE = "29 December 2021"
SUBTITLE = "8 AM 30 December 2021"
sts = utc(2021, 12, 29, 18)
ets = utc(2021, 12, 30, 20, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 50, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [8]:
cull = ['CIYI4', 'GLDI2']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [

]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=2)
res = mp.postprocess(filename='211230.png')
mp.close()

     state  wfo   val       lon      lat                                geo  \
0       MN  FGF  17.0  -94.9900  47.2700      POINT (377773.589 264197.989)   
1       CO  GJT  12.0 -107.7800  37.7000    POINT (-684655.559 -779304.905)   
2       ID  PIH  11.0 -111.5700  44.0000     POINT (-921591.196 -45460.723)   
3       AZ  FGZ  10.3 -109.9700  34.1500   POINT (-918990.705 -1150676.101)   
4       AZ  FGZ  10.1 -111.6700  35.1400  POINT (-1060936.086 -1021675.486)   
...    ...  ...   ...       ...      ...                                ...   
1407    MO  LSX   0.0  -90.6147  38.7256     POINT (813251.132 -651423.470)   
1408    MO  LSX   0.0  -90.5000  38.7833     POINT (822447.835 -643915.699)   
1409    MO  LSX   0.0  -91.8872  39.9714     POINT (690634.382 -525103.964)   
1410    MO  LSX   0.0  -92.0500  39.6833     POINT (679754.161 -558353.413)   
1411    MO  LSX   0.0  -90.5667  38.8000     POINT (816506.000 -642726.766)   

      used_for_analysis  nwsli  plotme source  
0  